**CLASSIFYING DOG BREEDS USING TRANFER LEARNING MODEL RESNET50V2.**

**HERE I HAVE TAKEN ONLY 10 SELECTED CLASSES,BUT THIS MODEL CAN BE USED FOR ENTIRE SET ALSO**


In [ ]:
#importing libraries
import numpy as np
import tensorflow as tf
from keras.applications.resnet_v2 import ResNet50V2
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#defining directories
train_path = '../input/dog-breed-identification/train'
test_path = '../input/dog-breed-identification/test'

#reading dataset labels
train_labels = pd.read_csv('../input/dog-breed-identification/labels.csv')
test_labels = pd.read_csv('../input/dog-breed-identification/sample_submission.csv')

In [ ]:
train_labels.head()

In [ ]:
test_labels.head()

In [ ]:
#in train_labels ids are not in jpg format, hence converting them to jpg
train_labels['id'] = train_labels['id'].apply(lambda x: x + ".jpg")
test_labels['id'] = test_labels['id'].apply(lambda x: x + ".jpg")

In [ ]:
#here i am training model for selected,training all breeds will take hours

x = train_labels.breed.unique()
include = ['beagle', 'chihuahua', 'doberman','french_bulldog', 'golden_retriever', 'malamute', 'pug', 'saint_bernard', 'scottish_deerhound','tibetan_mastiff']
for i in x :
    if i not in include:
        train_labels = train_labels.drop(train_labels[ train_labels['breed'] == i ].index)
print(train_labels.shape)
train_labels.head()

In [ ]:
plt.figure(figsize=(18,4))
cp = sns.countplot(x = 'breed', data = train_labels)


In [ ]:
#train_datagen is used for feature scaling and image augmentation (image augmentation is applied to avoid overfitting).
train_datagen = ImageDataGenerator(
                                   rescale = 1./255, 
                                   validation_split = 0.2,
                                   shear_range = 0.2, 
                                   zoom_range = 0.2, horizontal_flip = True, 
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   rotation_range = 20,
                                   brightness_range=[0.2,1.0])

#defining training set, here size of image is reduced to 224x224, batch of images is kept as 32 and class is defined as 'categorical'.
train_set = train_datagen.flow_from_dataframe(dataframe = train_labels,
                                                 directory = train_path,
                                                 x_col = "id",
                                                 y_col = "breed",
                                                 batch_size = 16,
                                                 subset="training",
                                                 class_mode = "categorical",
                                                 target_size = (224,224),
                                                 seed = 42,
                                                 shuffle = True)

In [ ]:
#defining validation set, here size of image is reduced to 224x224, batch of images is kept as 32 and class is defined as 'categorical' and subset as 'validation'.
validate_set = train_datagen.flow_from_dataframe(dataframe = train_labels,
                                                 directory = train_path,
                                                 x_col = "id",
                                                 y_col = "breed",
                                                 batch_size = 16,
                                                 subset="validation",
                                                 class_mode = "categorical",
                                                 target_size = (224,224),
                                                 seed = 42,
                                                 shuffle = True)

In [ ]:
#only rescaling is applied
test_datagen = ImageDataGenerator(rescale=1./255)

#defining test set
test_set = test_datagen.flow_from_dataframe(dataframe = test_labels,
                                                 directory = test_path,
                                                 x_col = "id",
                                                 y_col = None,
                                                 batch_size = 16,
                                                 class_mode = None,
                                                 seed = 42,
                                                 shuffle=False,
                                                 target_size = (224,224))

In [ ]:
train_set.batch_size

In [ ]:
1e-4

In [ ]:
#defining resnet50v2
resnet = ResNet50V2(input_shape = [224,224,3], weights = 'imagenet', include_top = False)

#we have to take pre trained weights, we don't want to train the weights again
for layer in resnet.layers:
    layer.trainable = False

#flattening the output of resnet for fully connected layer.
x = keras.layers.Flatten()(resnet.output)

#dropping out 40% to avoid overfitting
x = keras.layers.Dropout(0.4)(x)

#final layer will have 10 neurons
pred = keras.layers.Dense(10, activation='softmax')(x)

#creating the model 
model = tf.keras.models.Model(inputs=resnet.input, outputs=pred)

#taking learning rate as 0.00001
opt = tf.keras.optimizers.Adam(learning_rate = 1e-5)
model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])

#defining train step and validate step (total value/batch size)
train_step = train_set.n//train_set.batch_size
validate_step = validate_set.n//validate_set.batch_size

#training model
resnet50 = model.fit(train_set,validation_data = validate_set,epochs = 30,steps_per_epoch = train_step, validation_steps = validate_step)

In [ ]:
#evaluating model
model.evaluate(validate_set)

In [ ]:
#predicting output
STEP_SIZE_TEST=test_set.n//test_set.batch_size

#if we don't reset we'll get output in an unordered complex manner
test_set.reset()

pred=model.predict(test_set,steps=STEP_SIZE_TEST,verbose=1)

In [ ]:
plt.plot(resnet50.history['accuracy'])
plt.plot(resnet50.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

plt.plot(resnet50.history['loss'])
plt.plot(resnet50.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
#pred will be only probabilities, therefore take=ing class of highest probability.
predicted_class_indices=np.argmax(pred,axis=1)

#labels include class indices of train set
labels = (train_set.class_indices)

#creating a dictionery
labels = dict((v,k) for k,v in labels.items())

#predictions include names of predictions instead of number
predictions = [labels[k] for k in predicted_class_indices]

**NOTE:TRAINING AND VAILDATING WAS DONE FOR 10 CLASSES ONLY BUT WE ARE TESTING IT ON DATASET CONTAINING ALL THE CLASSES(TOTOAL 120).**

**THEREFORE WE'LL BE GETTING BAD VALUES FOR THE CLASSES WHICH AREN'T TRAINED**